In [79]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
import logging

#model building
import torch
import torchvision
from torchvision.transforms import v2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from time import sleep


#plotting and evalueation
import seaborn as sns


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [115]:
def pre_preprocessing(train_path, test_path, plot = True):

    #reading data 
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    #Stripping the whitespace from columns 
    train_df.rename(columns = lambda x: x.strip(), inplace = True)
    test_df.rename(columns = lambda x: x.strip(), inplace = True)

    #Subtracting 1 from each value in target column to scale to 0 or1 
    y_train = (train_df["TARGET"]-1).to_numpy()

    
    #Saving the ID values
    train_ids = train_df["ID"]
    test_ids = test_df["ID"]

    #Dropping the ID colmn
    train_df = train_df.drop('ID', axis = 1)
    test_df = test_df.drop('ID', axis = 1)

    #Creating some additional features

    #Normalized Difference Vegetation Index
    train_df["NDVI"] = (train_df["nir_p50"] - train_df["red_p50"])/ (test_df["nir_p50"] + test_df["red_p50"]) 
    #Normalized Difference Water Index
    train_df["NDWI"] = (train_df["nir_p50"] - train_df["swir1_p50"])/ (test_df["nir_p50"] + test_df["swir1_p50"])

    train_df["red_green_ratio"] = train_df["red_p50"]/train_df["green_p50"]
    train_df["NIR_green_ratio"] = train_df["nir_p50"]/train_df["green_p50"]

    train_df["blue_red_ratio"] = train_df["blue_p50"]/train_df["red_p50"]
    train_df["swir_ratio"] = train_df["swir1_p50"]/train_df["swir1_p50"]

    train_df["VV_VH_ratio"] = train_df["VV_p50"]/train_df["VH_p50"]


    #convering to Numpy arrays
    x_train = train_df.to_numpy()
    x_test = test_df.to_numpy()

    #normalizing the data 
    x_train = preprocessing.normalize(x_train)
    x_test = preprocessing.normalize(x_test)

    #Converting to Torch Tensors
    x_train = torch.from_numpy(x_train).type(torch.float)
    x_test = torch.from_numpy(x_test).type(torch.float)
    y_train = torch.from_numpy(y_train).type(torch.float)

    return x_train, y_train,x_test, test_ids

**Custom Dataset Class**

In [13]:
#Defining Custom Dataset
class CountryDataset(Dataset):
    def __init__(self, x, y):
        self.features = x
        self.labels = y

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self. labels[idx]


In [14]:
#appending all data framees

# Convert DataFrames to tensors before concatenation

**Instansiating Custom Datasets**

In [114]:
kenya_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv")
kenya_df.columns

Index(['ID  ', 'lon         ', 'lat         ', 'blue_p50 ', 'green_p50 ',
       'nir_p50 ', 'nira_p50 ', 're1_p50 ', 're2_p50 ', 're3_p50 ', 'red_p50 ',
       'swir1_p50 ', 'swir2_p50 ', 'VV_p50      ', 'VH_p50      ', 'TARGET'],
      dtype='object')

In [116]:
x_train_kenya, y_train_kenya,kenya_test_df, kenya_test_ids = pre_preprocessing(train_path = "geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv",
                                                                                test_path = "geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_testing.csv")

In [117]:
print(x_train_kenya.size())

print(y_train_kenya.size())

torch.Size([998, 22])
torch.Size([998])


In [118]:
batch_size = 32

#creating custom Datasets
train_dataset_kenya = CountryDataset(x_train_kenya, y_train_kenya)




#creating custom DataLoaders
kenya_train_dl = DataLoader(train_dataset_kenya, batch_size = batch_size, shuffle = True)

**Model Class**

In [72]:
#Creating the Module
class First_Model(nn.Module,):
    def __init__(self):
        super(First_Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(22, 128),  # Input size 22features
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),   # Output size is 1 for binary classification
            nn.Sigmoid()         # Use Sigmoid for binary classification
        )

    def forward(self, x):
        return self.net(x)


**Function for training the model**

In [101]:
def training(model, loss_fn, train_loader, opt, epochs = 1000):

    running_losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0 
        for i, (datapoints, labels) in enumerate(tqdm(train_loader)):


            #forward proporgation
            outputs = model(datapoints)


            labels = labels.float().unsqueeze(1)
            loss = loss_fn(outputs, labels)

            #backpropogation
            opt.zero_grad()
            loss.backward()
            opt.step()


            running_losses.append(loss.item())
            epoch_loss+=loss.item()

            # if (i+1) %10 == 0:
            #     print(f"epoch number {epoch+1}, loss = {loss.item()}")
    print("Training Finished")
    return running_losses


**Function to Calculate Accuracy of the Model**

In [ ]:
def calculate_accuracy(model, test_loader):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():
        for datapoints, labels in test_loader:
            outputs = model(datapoints)

            #values between 0 and 1
            probabilities = torch.sigmoid(outputs)

            #binary classification
            predicted = (probabilities > 0.5).float()

            #adjusting shape
            labels = labels.unsqueeze(1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)


    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    return accuracy

In [88]:
print(x_train_kenya.shape[1])

22


In [ ]:
n_features = x_train_kenya.shape[1]
model = First_Model()
loss_function  = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

running_losses = training(model = model, loss_fn = loss_function, train_loader = kenya_train_dl, opt = optimizer)

accuracy =  calculate_accuracy(model, kenya_train_dl)

100%|██████████| 32/32 [00:00<00:00, 1616.63it/s]

Training Finished
Test Accuracy: 80.86172344689379%
